In [ ]:
import os
from agents import GeneralAgent, PerceiveAgent, LearnAgent, ActionAgent, CriticAgent, ReflectAgent, get_agents
from assistant import Assistant
from library import Library
import random
from utils import SCORE_MAP, make_name

In [ ]:
prompt_dir = "./prompt"
library_path = "./library"
item_library_path = os.path.join(library_path, "item")
record_library_path = os.path.join(library_path, "record")    
personality_library_path = os.path.join(library_path, "personality")
history_library_path = "./examples"

# check exist and create
for path in [library_path, item_library_path, record_library_path, personality_library_path]:
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
library = Library(item_library_path, record_library_path, personality_library_path, history_library_path)
perceive_agent, learn_agent, action_agent, reflect_agent, critic_agent = get_agents(prompt_dir = prompt_dir)

In [ ]:
cur_user_id = random.choice(list(library.history_dict.keys()))
cur_user_history = library.history_dict[cur_user_id]
assistant = Assistant(perceive_agent, learn_agent, action_agent, reflect_agent, critic_agent, cur_user_id, library)

In [ ]:
domain = random.choice(["Movies_and_TV", "Books", "Video_Games"])

# can learn from more histories for better personalities 
learn1, learn2 = random.choices(cur_user_history["learn"][domain], k=2)
proxy = random.choice(cur_user_history["proxy"][domain])
unseen = random.choice(cur_user_history["unseen"][domain])

# learn-act-critic loop

In [ ]:
learn1_response = assistant.step_learn_act_critic(
        item_id=learn1["asin"], item_name=make_name(learn1["title"], domain),
        user_action=SCORE_MAP[learn1["overall"]], user_comment=learn1["reviewText"], max_try_times=2, log=True)

In [ ]:
# directly add
new_prefer = learn1_response["result"]["new_personality"]["prefer"]
new_disprefer = learn1_response["result"]["new_personality"]["disprefer"]

assistant.prefer += new_prefer
assistant.disprefer += new_disprefer


In [ ]:
learn2_response = assistant.step_learn_act_critic(
        item_id=learn2["asin"], item_name=make_name(learn2["title"], domain),
        user_action=SCORE_MAP[learn2["overall"]], user_comment=learn2["reviewText"], max_try_times=2, log=True)

# reflection mechanism

In [ ]:
# with reflection
new_prefer = learn2_response["result"]["new_personality"]["prefer"]
new_disprefer = learn2_response["result"]["new_personality"]["disprefer"]
reflect_response = assistant.reflect(new_prefer=new_prefer, new_disprefer=new_disprefer, log=True)

# make actions

In [ ]:
assistant.act(item_id=proxy["asin"], item_name=make_name(proxy["title"], domain))

In [ ]:
assistant.act(item_id=unseen["asin"], item_name=make_name(unseen["title"], domain))